In [823]:
# carregar as bibliotecas necessárias

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tabulate import tabulate

In [824]:
# ler cada uma das sheets e criar um dataframe para cada uma delas

df_INPUT = pd.read_excel("Input.xlsx", sheet_name="Planilha1")

# imprime o número de linhas e colunas do dataframe
print(df_INPUT.shape)

# apresentar as primeiras 5 linhas do dataframe "df_INPUT"
df_INPUT.head()

(1, 36)


,PC1,PC2,PC3,PC4,PC5,QC1,QC2,QC3,QC4,QC5,...,Rz45,Xz12,Xz13,Xz14,Xz23,Xz45,Sb,Ub2,PG1,QG1
0,80,90,60,50,50,20,20,10,10,10,...,0.9,18,18,9,9,9,100,150,20,10


In [825]:
# Criando o dataset para potências consumidas (PC1 a PC5)
df_potencias_consumidas = df_INPUT.filter(regex='^PC')

# Criando o dataset para potências reativas consumidas (QC1 a QC5)
df_potencias_reativas = df_INPUT.filter(regex='^QC')

# Criando o dataset para resistências de linha (Rz12 a Rz45)
df_resistencias_linhas = df_INPUT.filter(regex='^Rz')

# Criando o dataset para reatâncias de linha (Xz12 a Xz45)
df_reatancias_linhas = df_INPUT.filter(regex='^Xz')

# Exemplo de como visualizar um dos datasets criados
print("Dataset de Potências Consumidas:")
print(df_potencias_consumidas)

print("\nDataset de Potências Reativas Consumidas:")
print(df_potencias_reativas)

print("\nDataset de Resistências de Linha:")
print(df_resistencias_linhas)

print("\nDataset de Reatâncias de Linha:")
print(df_reatancias_linhas)

Dataset de Potências Consumidas:
   PC1  PC2  PC3  PC4  PC5
0   80   90   60   50   50

Dataset de Potências Reativas Consumidas:
   QC1  QC2  QC3  QC4  QC5
0   20   20   10   10   10

Dataset de Resistências de Linha:
   Rz12  Rz13  Rz14  Rz23  Rz45
0   1.8   1.8   0.9   0.9   0.9

Dataset de Reatâncias de Linha:
   Xz12  Xz13  Xz14  Xz23  Xz45
0    18    18     9     9     9


In [826]:
def create_variables_from_excel(file_path):
    # Ler o arquivo Excel
    df = pd.read_excel(file_path)
    
    # Garantir que o DataFrame tem apenas uma linha de dados
    if df.shape[0] != 1:
        raise ValueError("O arquivo deve conter exatamente uma linha de dados.")
    
    # Extrair a primeira linha de dados
    data_row = df.iloc[0]
    
    # Criar uma variável para cada coluna
    variables = {}
    for col_name in df.columns:
        variables[col_name] = data_row[col_name]
    
    return variables

file_path = 'Input.xlsx'

variables = create_variables_from_excel(file_path)

print(variables['PC1'])

80.0


In [827]:
# Cálculo das zonas de tensão

Sb = variables["Sb"]

Ub2 = variables["Ub2"]
Ub1 = (variables["PriT1"] / variables["SecT1"]) * variables["Ub2"]
print(Ub1)

Ub3 = (variables["PriT2"] / variables["SecT2"]) * variables["Ub2"]
print(Ub3)

Ub4 = round((variables["PriT3"] / variables["SecT3"]) * variables["Ub2"], 2)
print(Ub4)

df_pu = pd.DataFrame({
    "Sb": [Sb],
    "Ub1": [Ub1],
    "Ub2": [Ub2],
    "Ub3": [Ub3],
    "Ub4": [Ub4]
})

df_pu.head()


60.0
60.0
14.8


,Sb,Ub1,Ub2,Ub3,Ub4
0,100.0,60.0,150.0,60.0,14.8


In [828]:
# Cálculos transformadores

X1 = complex(0, (variables["xT1"] / 100) * (variables["Sb"] / variables["ST1"]) * (variables["PriT1"] / Ub1) ** 2)
print(X1) 

X2 = complex(0, (variables["xT2"] / 100) * (variables["Sb"] / variables["ST2"]) * (variables["PriT2"] / Ub3) ** 2)
print(X2) 

X3 = complex(0, round( (variables["xT3"] / 100) * (variables["Sb"] / variables["ST3"]) * (variables["PriT3"] / Ub4) ** 2, 4))
print(X3) 

df_pu["X1"] = X1
df_pu["X2"] = X2
df_pu["X3"] = X3

# Exibe as primeiras linhas do DataFrame atualizado
print(df_pu.head())

0.03j
0.03j
0.0411j
      Sb   Ub1    Ub2   Ub3   Ub4          X1          X2              X3
0  100.0  60.0  150.0  60.0  14.8  0.00+0.03j  0.00+0.03j  0.0000+0.0411j


In [829]:
# Cálculo das linhas

Zb2 = (variables["Ub2"]) ** 2 / variables["Sb"]
print(Zb2)

Z12 = complex(variables["Rz12"] , variables["Xz12"]) / Zb2
Z13 = complex(variables["Rz13"] , variables["Xz13"]) / Zb2
Z14 = complex(variables["Rz14"] , variables["Xz14"]) / Zb2
Z23 = complex(variables["Rz23"] , variables["Xz23"]) / Zb2
Z45 = complex(variables["Rz45"] , variables["Xz45"]) / Zb2
print(Z12)
print(Z13)
print(Z14)
print(Z23)
print(Z45)

# Carga líquida C1 = C1 original - produção G1 (fixa)
P_C1 = complex(variables["PC1"] , variables["QC1"]) - complex(variables["PG1"] , variables["QG1"]) 
print(P_C1)

df_pu["Z12"] = Z12
df_pu["Z13"] = Z13
df_pu["Z14"] = Z14
df_pu["Z23"] = Z23
df_pu["Z45"] = Z45
df_pu["P_C1"] = round(P_C1.real, 0) / Sb
df_pu["Q_C1"] = round(P_C1.imag, 0) / Sb

# Exibe as primeiras linhas do DataFrame atualizado
print(df_pu.head())

225.0
(0.008+0.08j)
(0.008+0.08j)
(0.004+0.04j)
(0.004+0.04j)
(0.004+0.04j)
(60+10j)
      Sb   Ub1    Ub2   Ub3   Ub4          X1          X2              X3  \
0  100.0  60.0  150.0  60.0  14.8  0.00+0.03j  0.00+0.03j  0.0000+0.0411j   

            Z12           Z13           Z14           Z23           Z45  P_C1  \
0  0.008+0.080j  0.008+0.080j  0.004+0.040j  0.004+0.040j  0.004+0.040j   0.6   

   Q_C1  
0   0.1  


In [830]:
df_potencias_consumidas_pu = df_potencias_consumidas / Sb
print(df_potencias_consumidas_pu)

df_pu = pd.concat([df_pu, df_potencias_consumidas_pu], axis=1)

df_potencias_reativas_pu = df_potencias_reativas / Sb
print(df_potencias_consumidas_pu)

df_pu = pd.concat([df_pu, df_potencias_reativas_pu], axis=1)

df_pu.head()

   PC1  PC2  PC3  PC4  PC5
0  0.8  0.9  0.6  0.5  0.5
   PC1  PC2  PC3  PC4  PC5
0  0.8  0.9  0.6  0.5  0.5


,Sb,Ub1,Ub2,Ub3,Ub4,X1,X2,X3,Z12,Z13,...,PC1,PC2,PC3,PC4,PC5,QC1,QC2,QC3,QC4,QC5
0,100.0,60.0,150.0,60.0,14.8,0.00+0.03j,0.00+0.03j,0.0000+0.0411j,0.008+0.080j,0.008+0.080j,...,0.8,0.9,0.6,0.5,0.5,0.2,0.2,0.1,0.1,0.1


In [831]:
df_class = pd.DataFrame({"Barramento": [1, 2, 3, 4, 5], 
                   "Pc": [np.nan, np.nan, np.nan, np.nan, np.nan],
                   "Qc": [np.nan, np.nan, np.nan, np.nan, np.nan],
                   "Pg": [np.nan, np.nan, np.nan, np.nan, np.nan],
                   "Qg": [np.nan, np.nan, np.nan, np.nan, np.nan],
                   "Pi": [np.nan, np.nan, np.nan, np.nan, np.nan],
                   "Qi": [np.nan, np.nan, np.nan, np.nan, np.nan],
                   "V": [np.nan, np.nan, np.nan, np.nan, np.nan],
                   "θ": [np.nan, np.nan, np.nan, np.nan, np.nan],
                   "Class.": [np.nan, np.nan, np.nan, np.nan, np.nan]})

for i, col in enumerate(df_potencias_consumidas.columns):
    df_class.at[i, "Pc"] = df_potencias_consumidas_pu[col][0]

for i, col in enumerate(df_potencias_reativas.columns):
    df_class.at[i, "Qc"] = df_potencias_reativas_pu[col][0]

df_class.head()

,Barramento,Pc,Qc,Pg,Qg,Pi,Qi,V,θ,Class.
0,1,0.8,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,0.9,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,0.6,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,0.5,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,0.5,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [832]:
# Trânsito de potências

#B1 barramento de referencia das fases 
θB1 = 0

#PG1 
PG1 = variables["PG1"] / variables["Sb"]
print("PG1 =", PG1)

#PG2 = 120 MW
PG2 = 120 / variables["Sb"]
print("PG2 =", PG2)

# Redução à parte emalhada
#PG3 = PCarga Total - PG2 - PG1
PG3 = round((variables["PC1"] + variables["PC2"] + variables["PC3"] + variables["PC4"] + variables["PC5"])/ variables["Sb"] - PG2 - variables["PG1"]/ variables["Sb"], 1)
print("PG3 =", PG3)

B11 = (1 / Z12.imag) + (1 / Z13.imag)
B12 = -1 / Z12.imag
B13 = -1 / Z13.imag
B21 = -1 / Z12.imag
B22 = (1 / Z12.imag) + (1 / Z23.imag)
B23 = -1 / Z23.imag
B31 = -1 / Z13.imag
B32 = -1 / Z23.imag
B33 = (1 / Z13.imag) + (1 / Z23.imag)

matriz_B = np.matrix([[B11, B12, B13], 
            [B21, B22, B23],
            [B31, B32, B33]])

print("Matrix B =", matriz_B)

# Como #B1 barramento de referencia das fases então
new_matrix_B = matriz_B[1:, 1:]

print("Nova Matriz B =", new_matrix_B)

P14 = variables["PC1"]/ variables["Sb"] + variables["PC4"]/ variables["Sb"] - PG1
P45 = variables["PC5"]/ variables["Sb"]
P2 = PG2 - variables["PC2"]/ variables["Sb"]
P3 = PG3 - variables["PC3"]/ variables["Sb"]

matriz_P = np.array([[P2], 
                     [P3]])

print("Matrix P =", matriz_P)


matriz_θ = np.linalg.inv(new_matrix_B) @ matriz_P

print("Matrix θ =", matriz_θ)

θB2 = round(matriz_θ[0, 0], 3)  # Acessa o primeiro elemento
print("θB2 =", θB2)

θB3 = round(matriz_θ[1, 0], 3)  # Acessa o primeiro elemento
print("θB3 =", θB3)

# Cálculos para a parte emalhada
P12 = (θB1 - θB2) / Z12.imag
print(P12)

P13 = round((θB1 - θB3) / Z13.imag, 2)
print(P13)

P23 = round((θB2 - θB3) / Z23.imag, 2)
print(P23)

# Cálculos para a parte radial
θB4 = round(-1 * (P14 * Z14.imag), 3)
print(θB4)

θB5 = round(-1 * ((P45 * Z45.imag) - θB4), 3)
print(θB5)

df_pu["PG1"] = PG1
df_pu["PG2"] = PG2
df_pu["PG3"] = PG3
df_pu["P12"] = P12
df_pu["P13"] = P13
df_pu["P23"] = P23
df_pu["P14"] = P14
df_pu["P45"] = P45
df_pu["θB1"] = θB1
df_pu["θB2"] = θB2
df_pu["θB3"] = θB3
df_pu["θB4"] = θB4
df_pu["θB5"] = θB5

# Exibe as primeiras linhas do DataFrame atualizado
df_pu.head()

PG1 = 0.2
PG2 = 1.2
PG3 = 1.9
Matrix B = [[ 25.  -12.5 -12.5]
 [-12.5  37.5 -25. ]
 [-12.5 -25.   37.5]]
Nova Matriz B = [[ 37.5 -25. ]
 [-25.   37.5]]
Matrix P = [[0.3]
 [1.3]]
Matrix θ = [[0.056]
 [0.072]]
θB2 = 0.056
θB3 = 0.072
-0.7
-0.9
-0.4
-0.044
-0.064


,Sb,Ub1,Ub2,Ub3,Ub4,X1,X2,X3,Z12,Z13,...,P12,P13,P23,P14,P45,θB1,θB2,θB3,θB4,θB5
0,100.0,60.0,150.0,60.0,14.8,0.00+0.03j,0.00+0.03j,0.0000+0.0411j,0.008+0.080j,0.008+0.080j,...,-0.7,-0.9,-0.4,1.1,0.5,0,0.056,0.072,-0.044,-0.064
